# TP: Spatial Linear Interference Cancellation

#### ___Adaptation of the Spatial ICMF via___
---

<img src=images/ICMF_via_LMS.png width='' height='' > </img>

So consider the adaptation of the spatial Interference Canceling Matched Filter (ICMF) depicted in the figure above, which in the spatial case is also called Generalized Sidelobe Canceller (Lecture 1, slides 61-63; see also the following slides on multi-user receivers). The received signal $y[k]$ contains $m$ subchannels and the interference canceling (IC) filter $\mathbf{f}$ is represented as a row vector. For a generic value of $\mathbf{f}$, we get the error signal

$$
\begin{equation}
e[k](\mathbf{f}) = d[k] - \mathbf{f} \mathbf{x}[k]
\end{equation}
$$

(the term ”epsilon” in Matlab is reserved for the precision of the Matlab implementation, hence do not use that for a variable name). For the adaptation of $\mathbf{f}$, the error signal is $\epsilon[k]$ which also provides an estimate of the transmitted symbol sequence $a[k]$ (up to a scale factor, which is $\|\mathbf{h}\|^2$ when $\mathbf{h}$ is known perfectly)), the desired response signal is $d[k]$, which is the spatial matched filter output $\mathbf{y}_1[k]$, and the input signal is $\mathbf{x}[k]$, which is also the output $\mathbf{y}_2[k]$ of the orthogonal complement filter $\mathbf{h}^{\perp H}$.

We shall consider the following signal scenario. The signal of interest is a QPSK signal, $a[k] = \frac{\sigma_a}{\sqrt(2)} (\pm 1 \pm j)$ of which the real and imaginary parts are generated as pseudorandom binary sequences. We shall use $\sigma_a$ to represent the propagation attenuation also. This signal gets received by a uniform linear array (ULA) so that $\mathbf{h} = \mathbf{a}(\theta)$ where the ULA response is of the form

$$
\begin{equation}
\mathbf{a}(\theta) = \begin{bmatrix}
       1 \\
       e^{j2\pi \frac{d}{\lambda} \sin(\theta)} \\[0.3em]
       \vdots \\
       e^{j(m-1)2\pi \frac{d}{\lambda} \sin(\theta)} \\[0.3em]
     \end{bmatrix}
\end{equation}
$$